In [27]:
import numpy
import pandas
import matplotlib.pyplot as plt
import seaborn
import scipy.stats

import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing

import torch.nn as nn
import torch.optim as optim
import torch as ptr

In [28]:
# we are desinging a BI-LSTM model to predict the stock price of a company
# we will use the past 60 days stock price to predict the next day stock price
# we will use the stock price of ALT.csx for this task

# load the data
data = pandas.read_csv('data/ALT (1).csv')
data = data.dropna()
data = data[['open', 'high', 'low', 'close','volume','trade_count']]
data = data.values

# split the data into training and testing data
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

# scale the data
scaler = sklearn.preprocessing.MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# create the training data
X_train = []
y_train = []
for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i])
    y_train.append(train_data[i])
X_train = numpy.array(X_train)
y_train = numpy.array(y_train)

# create the testing data
X_test = []
y_test = []
for i in range(60, len(test_data)):
    X_test.append(test_data[i-60:i])
    y_test.append(test_data[i])
X_test = numpy.array(X_test)
y_test = numpy.array(y_test)

# Training and testing code
X_train = torch.tensor(X_train).float().to(device)
y_train = torch.tensor(y_train).float().to(device)
X_test = torch.tensor(X_test).float().to(device)
y_test = torch.tensor(y_test).float().to(device)


In [ ]:
# create the model
# Create a Bidirectional LSTM model with 5 Bi directional LSTM layers with 128, units for 3 layers and 2  layers with 64 units
# the output layer will have 4 units
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(CustomLSTM, self).__init__()
        self.hidden_sizes = hidden_sizes
        self.num_layers = len(hidden_sizes)
        
        # Define LSTM layers
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        self.lstm4 = nn.LSTM(hidden_sizes[2], hidden_sizes[3], batch_first=True)
        self.lstm5 = nn.LSTM(hidden_sizes[3], hidden_sizes[4], batch_first=True)
        
        # Define fully connected layer
        self.fc = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        h0_1 = torch.zeros(1, x.size(0), self.hidden_sizes[0]).to(device)
        c0_1 = torch.zeros(1, x.size(0), self.hidden_sizes[0]).to(device)
        out, _ = self.lstm1(x, (h0_1, c0_1))
        
        h0_2 = torch.zeros(1, x.size(0), self.hidden_sizes[1]).to(device)
        c0_2 = torch.zeros(1, x.size(0), self.hidden_sizes[1]).to(device)
        out, _ = self.lstm2(out, (h0_2, c0_2))
        
        h0_3 = torch.zeros(1, x.size(0), self.hidden_sizes[2]).to(device)
        c0_3 = torch.zeros(1, x.size(0), self.hidden_sizes[2]).to(device)
        out, _ = self.lstm3(out, (h0_3, c0_3))
        
        h0_4 = torch.zeros(1, x.size(0), self.hidden_sizes[3]).to(device)
        c0_4 = torch.zeros(1, x.size(0), self.hidden_sizes[3]).to(device)
        out, _ = self.lstm4(out, (h0_4, c0_4))
        
        h0_5 = torch.zeros(1, x.size(0), self.hidden_sizes[4]).to(device)
        c0_5 = torch.zeros(1, x.size(0), self.hidden_sizes[4]).to(device)
        out, _ = self.lstm5(out, (h0_5, c0_5))
        
        out = self.fc(out[:, -1, :])
        return out
    
# Example usage:
input_size = 6  # Example input size
hidden_sizes = [128, 128, 128, 64, 64]  # Example hidden sizes
output_size = 1  # Example output size
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CustomLSTM(input_size, hidden_sizes, output_size).to(device)

In [32]:
print(model)

CustomLSTM(
  (lstm1): LSTM(6, 128, batch_first=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (lstm3): LSTM(128, 128, batch_first=True)
  (lstm4): LSTM(128, 64, batch_first=True)
  (lstm5): LSTM(64, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [34]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'epoch {epoch}: loss {loss.item()}')

# Test the model
model.eval()
outputs = model(X_test)
loss = criterion(outputs, y_test)
print(f'test loss {loss.item()}')

# Plot the results
plt.figure(figsize=(14, 5))
plt.plot(scaler.inverse_transform(y_test.cpu().detach().numpy()), label='True')
plt.plot(scaler.inverse_transform(outputs.cpu().detach().numpy()), label='Predicted')
plt.legend()
plt.show()


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.08 GiB. GPU 0 has a total capacity of 7.75 GiB of which 5.81 GiB is free. Including non-PyTorch memory, this process has 1.17 GiB memory in use. Of the allocated memory 1.03 GiB is allocated by PyTorch, and 37.42 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)